<h2>Market Study - Chicken Exports</h2>

<a href="https://openclassrooms.com/en/" >Openclassrooms</a>, Data Analyst Course
<br>Project 5 - Michael Orange


<hr>

A chicken producer is looking for new export markets. All countries are considered. 
<br>The core objective is to assist the company in targeting the markets with the highest potential. 

'Grouping' the countries in 'clusters' with similar caracteristics is expected. Variables are: 
- country population, 
- population growth, 
- calory supply per capita and protein supply per capita, 
- prevalence of animal proteins in the mix. 

Additional relevant variable might be included (ex. GDP per capita)

<hr>

<h2>Preparation Dataset data_country</h2>

<h3>Import Datasets</h3>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

Data are collected from The Food and Agriculture Organization Corporate Statistical Database (FAOSTAT). The FAOSTAT website disseminates statistical data collected and maintained by the Food and Agriculture Organization (FAO).

FAOSTAT - Data collected (2013 and 2017 are selected for the below analysis): analysis is based on 2017 figures, with a comparaison from 2013 data. 
2017 is the most recent comprehensive data from FAOSTAT.

- Datasets Food Balance Animal and Vegetal
- Dataset Population
- Dataset Gross Domestic Product (GDP)
- Dataset Trade Mix - Chicken 

In [2]:
#import food datasets
veg2013 = pd.read_csv("data/raw/vegetal2013.csv")
ani2013 = pd.read_csv("data/raw/animal2013.csv")
veg2017 = pd.read_csv('data/raw/vegetal2017.csv', dtype={'Note': str })
ani2017 = pd.read_csv("data/raw/animal2017.csv")
ani2013["origin"] = "animal"
veg2013["origin"] = "vegetal"
ani2017["origin"] = "animal"
veg2017["origin"] = "vegetal"

#import population
pop = pd.read_csv("data/raw/FAOSTAT_data_Population_2013-2017.csv")

# import GDP and Stability
gdp = pd.read_csv("data/raw/FAOSTAT_data_MacroIndicators_20132017.csv")
stability = pd.read_csv("data/raw/FAOSTAT_data_PoliticalStability_2017.csv")

# import chicken trade
chicken_trade = pd.read_csv("data/raw/FAOSTAT_data_Trade_Chicken_20132017.csv")

# import chicken livestock
chicken_stock = pd.read_csv("data/raw/FAOSTAT_data_chicken_livestock_20132017.csv")

<h3>Preparation food dataset</h3>


In [3]:
# group food datatsets
temp = [veg2013, ani2013, ani2017, veg2017]
temp = pd.concat(temp, ignore_index=True)

# delete ani2013, veg2013, ani2017, veg2017
del ani2013, veg2013, ani2017, veg2017

temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element', 'item_code', 'item',
                'xx4', "year", "unit", "value", 'xx5', 'flag_description', 'origin', 'xx6']

data = temp.pivot_table(index=["year", "country_code", "country", "origin", "item_code", \
                               "item"], columns = ["element"], values=["value"], aggfunc=sum)

# rename columns
data.columns = ['domestic_supply_quantity','export_quantity','fat_supply_quantity_gcapitaday','feed',
                'food','food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr','import_quantity','losses','other_uses','processing',
                'production', 'protein_supply_quantity_gcapitaday', 'residuals', 'seed','stock_variation','tourist_consumption']

data = data.reset_index()

Data for China are duplicated. 
<br>data for 'China' = sum of the splitted data for 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan'.
- deletion of 'China' data (code 351)
- 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan' data are kept in order to preserve a better granularity.



In [4]:
# remove doublon from China (code_country 351)
data = data.loc[data.country_code != 351]

# remove Bermuda and Brunei - no information for 2017
data = data.loc[data.country != 'Bermuda']
data = data.loc[data.country != 'Brunei Darussalam']

<h3>Population data</h3>

- Population per country
- Growth of the population (%) - 2017 vs 2013

In [5]:
# population
pop.columns = ["xx","xx2","country_code","country", 'xx3', 'xx4', 'xx5','xx6', 
               'xx7', 'year',"xx8","population_total_thousand","xx9", 'xx10', 'xx11']

pop = pop.drop(["xx","xx2",'xx3','xx4','xx5', 'xx6', 'xx7','xx8', 'xx9', 'xx10', 'xx11'], axis = 1)
pop.reset_index(drop=True, inplace=True)

# merge data and pop
data = pd.merge(data, pop, how='left')

data_country = pop.pivot_table(index=['country_code', 'country'], columns = ['year'], values=['population_total_thousand'])
data_country.columns = ['pop_2013_thousand', 'pop_2017_thousand']
data_country.reset_index(inplace=True)

#growth population
data_country['pop_growth_percent'] = (data_country['pop_2017_thousand'] / data_country['pop_2013_thousand'] - 1) *100

In [6]:
#remove column not needed for the analysis
data_country.drop(['pop_2013_thousand'], axis = 1, inplace=True)

<h3>Diet caracterisitics : proteins and calories </h3>

- Food supply kcal per capita per day - per country
- Protein supply g per capita per day - per country
- Growth of the protein supply per capita per country (%) - 2017 vs 2013

In [7]:
temp = data.loc[data['year']==2017].pivot_table(index=['country_code', 'country'], \
                        values=['food_supply_kcalcapitaday',  'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['food_supply_kcalcapitaday_2017', 'protein_supply_gcapitaday_2017']
temp = temp.reset_index()

data_country = pd.merge(temp, data_country, how='left')

- Food supply kcal per country (kcal)
- Protein supply per country (kg)
- Growth of the protein supply per country (%) - 2017 vs 2013

**Animal proteins in the total protein supply**

- Protein supply kg from animals - per country
- Growth of the protein supply per country (%) from animals - 2017 vs 2013

In [8]:
NB_DAYS_YEAR = 365 

#total protein supply
data_country['protein_supply_kg_2017'] = \
    data_country['protein_supply_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

In [9]:
# protein supply from animals
temp = data.loc[(data['origin']=='animal') & (data['year']==2017)].pivot_table(index=['country_code', 'country'], \
                                                      values=[ 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['protein_supply_animal_gcapitaday_2017']
temp.reset_index(inplace=True)

data_country = pd.merge(data_country, temp, how='left')

In [10]:
# prevalence animal proteins in the total protein supply
data_country['protein_supply_animal_kg_2017'] = \
    data_country['protein_supply_animal_gcapitaday_2017']/1000 * NB_DAYS_YEAR *data_country['pop_2017_thousand']*1000

data_country['prevalence_protein_animal_percent_2017'] = \
    data_country['protein_supply_animal_kg_2017'] / data_country['protein_supply_kg_2017'] * 100

In [11]:
#remove column not needed for the analysis
data_country.drop(['protein_supply_animal_kg_2017', 'protein_supply_kg_2017', 'protein_supply_animal_gcapitaday_2017'], \
                  axis = 1, inplace=True)

<h3>Gross Domestic Product (GDP) per capita</h3>

- GDP per capita (USD)
- GDP Growth - 2017 vs 2013

In [12]:
gdp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "year", "xx7", "value", 'xx8', 'xx9', 'xx10']
gdp = gdp.loc[(gdp['item'] == 'Gross Domestic Product per capita') & (gdp['year']==2017)].pivot_table(\
                                                                                                      index=['country_code', "country"], columns = ['item'], values=['value'])
gdp.columns = ['gdp_percapita_usd_2017']
gdp.reset_index(inplace=True)

data_country = pd.merge(data_country, gdp, how='left')

GDP of Taiwan is missing. 
Imputation with information collected from the International Monetary Fund (IMF).
- GDP per capita 2017 : 50 593 USD

Source: https://bit.ly/2U2251b

In [13]:
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2017'] = 50593

<h3>Political Stability</h3>

In [14]:
stability.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "xx8", "xx8", "value", 'xx9', 'xx10', 'xx11']
stability = stability.pivot_table(index=['country_code', 'country'], columns = ['item'], values=['value'])
stability.columns = ['political_stability_index']
stability.reset_index(inplace=True)

data_country = pd.merge(data_country, stability, how='left')

Countries with no political stability index: {{data_country.loc[data_country['political_stability_index'].isna()].country.tolist()}}

- Imputation possible for the 2 French territories (French Polynesia and New Caledonia) with France's index.
- Imputation possible for Namibia with United Arab Emirates' index - the political stability index of the UAE is the closest (and almost equivalent) of Namibia in 2017 (source: World Bank - https://www.theglobaleconomy.com/rankings/wb_political_stability/)

In [15]:
# imputation French Polynesia and New Caledonia
for z in ['French Polynesia', 'New Caledonia']:
    data_country.loc[data_country['country'] == z, 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'France']['political_stability_index'].values

# imputation Namibia
data_country.loc[data_country['country'] == 'Namibia', 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'United Arab Emirates']['political_stability_index'].values

<h3>Poultry (with eggs)</h3>



Poultry are domesticated avian species that can be raised for eggs, meat and/or feathers. The term “poultry” covers a wide range of birds, from indigenous and commercial breeds of chickens to Muscovy ducks, mallard ducks, turkeys, guinea fowl, geese, quail, pigeons, ostriches and pheasants. 

- In 2017, chickens accounted for some 92 percent of the world’s poultry population, followed by ducks (5 percent), and turkeys (2 percent). 
- Chickens contribute 89 percent of world poultry meat production, followed by turkeys with 5 percent, ducks with 4 percent and geese and guinea fowl with 2 percent. The rest comes from other poultry species.
- Chickens provide 92 percent of world egg production.

source: http://www.fao.org/poultry-production-products/production/en/

There is no granular data about chicken exclusively in the dataset 'animal' but the data for the category 'Poultry' (in which chicken is accounting around 90% of the total) give us a fair indicator of the chicken market in each country. 

Poultry composition: 
- Meat chicken, Fat liver prepared (foie gras), Meat chicken canned, Meat,duck, Meat goose and guinea fowl, Meat turkey.
- Eggs are included

In [16]:
temp = data.loc[(data['item'] == 'Poultry Meat')|(data['item'] == 'Eggs')].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=['production', 'import_quantity'], aggfunc=sum)

temp.columns = ['production_poultry_tons_2013', 'production_poultry_tons_2017', 'import_poultry_tons_2013', 'import_poultry_tons_2017']

temp.reset_index(inplace=True)

#convert 1000 tons to tons
for z in temp.iloc[:, [2,3,4,5]].columns:
    temp[str(z)] = temp[str(z)] * 1000

data_country = pd.merge(data_country, temp, how='left')

In [17]:
#growth Poultry imports
data_country['import_poultry_growth'] = \
    np.where(data_country['import_poultry_tons_2013'] != 0, \
             (data_country['import_poultry_tons_2017'] / data_country['import_poultry_tons_2013'] - 1) * 100, 0)

In [18]:
#remove column not needed for the analysis
data_country.drop(['production_poultry_tons_2013', 'import_poultry_tons_2013'], axis = 1, inplace=True)

<h3>Chicken Importations (with eggs)</h3>

Data are from the food and agricultural trade dataset collected, processed and disseminated by FAO. The data is mainly provided by UNSD, Eurostat, and other national authorities as needed.
- Import quantity of 'chicken' per country
- Import value of 'chicken' per country

The category 'chicken' is made of 'chicken', 'meat, chicken', 'meat, chicken, canned', 'offals, liver chicken' + 'Eggs' were also included.
<br>Offals liver chicken was not available in the 'Poultry' section on the previous section.  

In [19]:
chicken_trade.columns =['xx', 'xx2', 'country_code', 'country', 'partner_country_code', 'partner_country', 'xx6', \
                        'element', 'xx8', 'xx9', 'xx10', 'year', 'unit', 'value', 'xx13', 'xx14', 'xx15']

# there is a disruptancy on the name of the UK between the FAO trade dataset and other FAO datasets
chicken_trade.loc[chicken_trade['country'] == 'United Kingdom', 'country'] = 'United Kingdom of Great Britain and Northern Ireland'
chicken_trade.loc[chicken_trade['partner_country'] == 'United Kingdom', 'partner_country'] = 'United Kingdom of Great Britain and Northern Ireland'

In [20]:
#Imports of chicken per country - in quantity and value

chicken_import = chicken_trade.loc[((chicken_trade['element'] == 'Import Value') | (chicken_trade['element'] == 'Import Quantity')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

chicken_import = chicken_import.pivot_table(index=["country_code", "country"], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_import.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons_2017', \
                         'import_chicken_value_thousand_usd_2017']
chicken_import.reset_index(inplace=True)

data_country = pd.merge(data_country, chicken_import, how='left')

In [21]:
country_no_report_import_chicken = \
    data_country.loc[(data_country['import_chicken_quantity_tons_2017'].isna()) | (data_country['import_chicken_quantity_tons_2013'].isna())]

{{ len(country_no_report_import_chicken)}} countries do have missing information about chicken importations due to the absence of reports in 2013 or 2017:

{{country_no_report_import_chicken.country.tolist()}}

We will get an approximation of these imports through the declaration of exports from partner countries.

In [22]:
# getting information about the chicken imports on the countries not reporting the imports through the export reports from other countries.

chicken_export = chicken_trade.loc[((chicken_trade['element'] == 'Export Quantity') | (chicken_trade['element'] == 'Export Value')) \
                                  & (chicken_trade['unit'] != '1000 Head')]

# Export of chicken to countries not reporting any chicken imports
chicken_export_to_countries_noreport = chicken_export.loc[chicken_export['partner_country'].isin(country_no_report_import_chicken.country)]

chicken_export_to_countries_noreport = chicken_export_to_countries_noreport.pivot_table(\
                                                                                       index=['partner_country_code', 'partner_country'], columns=['year', 'element'], values=['value'], aggfunc=sum)

chicken_export_to_countries_noreport.columns = ['import_chicken_quantity_tons_2013', 'import_chicken_value_thousand_usd_2013', \
                                                'import_chicken_quantity_tons_2017', 'import_chicken_value_thousand_usd_2017']
chicken_export_to_countries_noreport.reset_index(inplace=True)

chicken_export_to_countries_noreport.rename(columns = {'partner_country_code':'country_code', 'partner_country':'country'}, inplace=True)

In [23]:
data_country.set_index('country', inplace=True)
chicken_export_to_countries_noreport.set_index('country', inplace=True)

#overlapping values from chicken export to data_country for the countries not reporting their imports
data_country = data_country.combine_first(chicken_export_to_countries_noreport)
data_country.reset_index(inplace=True)

In [24]:
data_country['import_chicken_quantity_growth_percent'] = \
    (data_country['import_chicken_quantity_tons_2017'] / data_country['import_chicken_quantity_tons_2013'] - 1) *100

data_country['import_chicken_ratio_usd_tons_2017'] = \
    data_country['import_chicken_value_thousand_usd_2017'] * 1000 / data_country['import_chicken_quantity_tons_2017']

In [25]:
#remove column not needed for the analysis
data_country.drop(['import_chicken_value_thousand_usd_2013', 'import_chicken_quantity_tons_2013',\
                  'import_chicken_value_thousand_usd_2017'], axis = 1, inplace=True)

In [26]:
data_country

,country,country_code,food_supply_kcalcapitaday_2017,gdp_percapita_usd_2017,import_chicken_quantity_tons_2017,import_poultry_growth,import_poultry_tons_2017,political_stability_index,pop_2017_thousand,pop_growth_percent,prevalence_protein_animal_percent_2017,production_poultry_tons_2017,protein_supply_gcapitaday_2017,import_chicken_quantity_growth_percent,import_chicken_ratio_usd_tons_2017
0,Afghanistan,2,1997.0,468.297893,70001.0,9.090909,48000.0,-2.78,36296.113,12.477767,19.523017,83000.0,54.09,64.086637,1234.310938
1,Albania,3,3400.0,3347.701760,40034.0,-1.538462,64000.0,0.40,2884.169,-0.675703,55.497908,38000.0,119.50,66.669442,1314.332817
2,Algeria,4,3345.0,3264.338962,1991.0,4.724409,665000.0,-0.96,41389.189,8.518733,27.679052,2000.0,92.85,-26.774549,12734.304370
3,Angola,7,2266.0,2805.692595,267388.0,34.285714,47000.0,-0.29,29816.766,14.610305,30.449251,300000.0,54.09,-22.429454,1014.503269
4,Antigua and Barbuda,8,2429.0,10549.721325,6455.0,0.000000,0.0,0.76,95.426,4.272477,66.605052,7000.0,81.15,9.965928,1979.705655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Venezuela (Bolivarian Republic of),236,2247.0,6042.349852,25033.0,-32.088889,764000.0,-1.18,29402.484,-1.271131,45.115453,25000.0,56.30,-91.089620,2464.027484
168,Viet Nam,237,2938.0,1619.296641,363681.0,43.422354,1450000.0,0.31,94600.648,4.240161,40.875264,295000.0,90.03,-30.725185,1082.748343
169,Yemen,249,2062.0,515.814155,77952.0,6.363636,234000.0,-2.96,27834.819,10.687948,20.060105,78000.0,53.24,-28.257989,1554.674672
170,Zambia,251,2008.0,1115.460621,12070.0,0.000000,101000.0,0.11,16853.599,12.910256,19.776498,12000.0,59.06,669.279796,921.043911


In [27]:
data_country.columns

Index(['country', 'country_code', 'food_supply_kcalcapitaday_2017',
       'gdp_percapita_usd_2017', 'import_chicken_quantity_tons_2017',
       'import_poultry_growth', 'import_poultry_tons_2017',
       'political_stability_index', 'pop_2017_thousand', 'pop_growth_percent',
       'prevalence_protein_animal_percent_2017',
       'production_poultry_tons_2017', 'protein_supply_gcapitaday_2017',
       'import_chicken_quantity_growth_percent',
       'import_chicken_ratio_usd_tons_2017'],
      dtype='object')

In [28]:
data_country.isna().any()

country                                   False
country_code                              False
food_supply_kcalcapitaday_2017            False
gdp_percapita_usd_2017                    False
import_chicken_quantity_tons_2017         False
import_poultry_growth                     False
import_poultry_tons_2017                  False
political_stability_index                 False
pop_2017_thousand                         False
pop_growth_percent                        False
prevalence_protein_animal_percent_2017    False
production_poultry_tons_2017              False
protein_supply_gcapitaday_2017            False
import_chicken_quantity_growth_percent    False
import_chicken_ratio_usd_tons_2017        False
dtype: bool

<h2>Export Dataset data_country</h2>

In [29]:
data_country.to_csv(r'data/output/data_country.csv', index = False)